In [18]:
import pandas as pd
import os
import numpy as np
import scipy.sparse as sp
from tqdm.notebook import tqdm

dir_ = '../data/'
file_name = 'normalized_to_rating_filter_track_5_user_50.pkl'

In [19]:
lh = pd.read_pickle(os.path.join(dir_, file_name))
train = pd.read_pickle(os.path.join(dir_, 'train_' + file_name))
test = pd.read_pickle(os.path.join(dir_, 'test_' + file_name))

In [20]:
unique_users = len(lh['uid'].unique())
unique_tracks = len(lh['tid'].unique())
l = unique_users + unique_tracks + 19
tid_start = unique_users
tag_start = unique_users + unique_tracks
print ('Length of vector : ' + str(l))
print ('Tid start position : ' + str(tid_start))
print ('Tag start position : ' + str(tag_start))
print ('Number of unique users : ' + str(unique_users))
print ('Number of unique tracks : ' + str(unique_tracks))
print ('Times of tracks being listened: ' + str(len(lh)))
lh = None

Length of vector : 39374
Tid start position : 218
Tag start position : 39355
Number of unique users : 218
Number of unique tracks : 39137
Times of tracks being listened: 453347


In [21]:
i = 0
sp_rows = []
sp_cols = []
for index, row in tqdm(train.iterrows(), total=len(train)):
    sp_rows.append(i)
    sp_cols.append(row['uid'])
    
    sp_rows.append(i)
    sp_cols.append(tid_start + row['tid'])
    
    for tag in row['tags']:
        sp_rows.append(i)
        sp_cols.append(tag_start + tag)
        
    i += 1

In [22]:
max(sp_rows), max(sp_cols), len(train)

(362676, 39373, 362677)

In [23]:
w = np.ones(len(sp_rows))
train_sp = sp.csr_matrix((w, (sp_rows, sp_cols)), shape=(len(train), l), dtype=float)

In [24]:
train_sp.shape

(362677, 39374)

In [25]:
sp.save_npz('../data/train_x_'+ file_name, train_sp)

In [26]:
train_y = []
for index, row in tqdm(train.iterrows(), total=len(train)):
    train_y.append(row['rating'])

In [27]:
np.savetxt(os.path.join(dir_, 'train_y_' + file_name[:-3] + 'csv'), train_y, delimiter=",")

In [28]:
i = 0
sp_rows = []
sp_cols = []
for index, row in tqdm(test.iterrows(), total=len(test)):
    sp_rows.append(i)
    sp_cols.append(row['uid'])
    
    sp_rows.append(i)
    sp_cols.append(tid_start + row['tid'])
    
    for tag in row['tags']:
        sp_rows.append(i)
        sp_cols.append(tag_start + tag)

    i += 1

In [29]:
w = np.ones(len(sp_rows))
test_sp = sp.csr_matrix((w, (sp_rows, sp_cols)), shape=(len(test), l), dtype=float)

In [30]:
sp.save_npz('../data/test_x_' + file_name, test_sp)

In [31]:
test_y = []
for index, row in tqdm(test.iterrows(), total=len(test)):
    test_y.append(row['rating'])

In [32]:
np.savetxt(os.path.join(dir_, 'test_y_' + file_name[:-3] + 'csv'), test_y, delimiter=",")